# Задание 
1. Загрузить корпус текстовых данных с помощью DataFrame – бд Рубцовой (2 файла). ✅
2. Каждый файл сократить до 5000 строк. Объиденить 2 файла в один DaraFrame. ✅
3. Проверить пропущенные значения. Создать колонку с длиной текстов. Вычислить среднюю, медиану и моду.
4. Выполнить визуальный анализ текстового корпуса Рубцовой. Построить минимум 5 графиков, минимум по 1 из каждой библиотеки. Выбрать один график и полностью описать атрибуты используемого метода. 
5. Предобработать тексты, используя ЛР1.
6. Получить вектора признаков для каждого текста с помощью, BoW и TF-IDF.
7. Сохранить полученные вектора в файле.

## Раздел 0: Импорты библиотек и исходных данных
### 0.0 Импорты необходимых библиотек

In [1]:
import pandas as pd # импорт библиотеки для работы с DataFrame
import statistics # импорт библиотеки для вычисления статистических показателей
import re # импорт библиотеки для работы с регулярными выражениями
import nltk # импорт библиотеки NLTK
from nltk.tokenize import word_tokenize # импорт функции для токенизации 
# загрузка списков стоп слов 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/vadim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 0.1 Создаем таблицу для хранения данных в DataFrame

In [2]:
columns = [
    "id", "tdate", "tname", "ttext", "ttype", 
    "trep", "tfav", "tstcount", "tfol", 
    "tfrien", "listcount", "unknown"
]

positive_df = pd.read_csv("positive.csv", delimiter=";", names=columns, nrows=5000)
negative_df = pd.read_csv("negative.csv", delimiter=";", names=columns, nrows=5000)

df = pd.concat([positive_df, negative_df]).reset_index(drop=True)

print(f"(строки, колонки): {df.shape}")
df

(строки, колонки): (10000, 12)


,id,tdate,tname,ttext,ttype,trep,tfav,tstcount,tfol,tfrien,listcount,unknown
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,409766329734795264,1386530880,Anastasievnaa,Натааааш. у тебя было классное ДР =( и хрен у ...,-1,0,0,0,2002,30,21,0
9996,409766375364644864,1386530891,EmoKingSize,ой(( я совсем забыла восстановить старый аккау...,-1,0,0,0,5154,41,49,0
9997,409766474010886144,1386530915,yegorushki,Ну и где ночной твиттер? Все спят что ли? Так ...,-1,0,0,0,1456,48,56,0
9998,409766520009793536,1386530926,blond_4,"Девочка моя,спасибо тебе за этот вечер .мне оч...",-1,0,0,0,99,14,14,0


## Раздел 1: Анализ данных
### 1.0 Создание колонки с длинной текстов

In [3]:
# Создаем колонку с длинной текстов с помощью токенизации каждого ряда
df['tlength'] = df['ttext'].apply(lambda x: len(word_tokenize(str(x))))
df

,id,tdate,tname,ttext,ttype,trep,tfav,tstcount,tfol,tfrien,listcount,unknown,tlength
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,22
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,17
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,15
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,35
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,409766329734795264,1386530880,Anastasievnaa,Натааааш. у тебя было классное ДР =( и хрен у ...,-1,0,0,0,2002,30,21,0,16
9996,409766375364644864,1386530891,EmoKingSize,ой(( я совсем забыла восстановить старый аккау...,-1,0,0,0,5154,41,49,0,23
9997,409766474010886144,1386530915,yegorushki,Ну и где ночной твиттер? Все спят что ли? Так ...,-1,0,0,0,1456,48,56,0,27
9998,409766520009793536,1386530926,blond_4,"Девочка моя,спасибо тебе за этот вечер .мне оч...",-1,0,0,0,99,14,14,0,18


### 1.1 Проверка на пропущенные значения - ПРОВЕРИТЬ!!!

In [4]:
# Суммируем все пропущенные значения в каждой колонке
df.isnull().sum()

id           0
tdate        0
tname        0
ttext        0
ttype        0
trep         0
tfav         0
tstcount     0
tfol         0
tfrien       0
listcount    0
unknown      0
tlength      0
dtype: int64

### 1.2 Вычисление средней, медианы и моды

In [5]:
# Выводим среднее, медиану и моду длин текстов с помощью библеотеки statistics
print(f'Средняя длина текстов: {statistics.mean(df["tlength"])}\n'
      f'Медиана длины текстов: {statistics.median(df["tlength"])}\n'
      f'Мода длины текстов: {statistics.mode(df["tlength"])}')

Средняя длина текстов: 17.6985
Медиана длины текстов: 16.0
Мода длины текстов: 13


## Раздел 2: Визуальный анализ текстового корпуса Рубцовой
### 2.0 Построение графика

## Раздел 3: Предобработка текстов
### 3.0 Приведение к нижнему регистру, токенизация и леминизация

In [6]:
# Приводим все тексты к нижнему регистру  
df['ttext'] = df['ttext'].str.lower()
# Удаляем символы, английские слова и стоп-слова и осуществляем токенизацию
stopwords= nltk.corpus.stopwords.words('russian')
df['ttext'] = df['ttext'].apply(lambda x: [word for word in word_tokenize(str(x)) if re.match(r'[а-я]', word) and word not in stopwords])
morph = pymorphy2.MorphAnalyzer()
df

,id,tdate,tname,ttext,ttype,trep,tfav,tstcount,tfol,tfrien,listcount,unknown,tlength
0,408906692374446080,1386325927,pleease_shut_up,"[школота, поверь, самое, общество, профилирующ...",1,0,0,0,7569,62,61,0,22
1,408906692693221377,1386325927,alinakirpicheva,"[все-таки, немного, похож, мальчик, равно]",1,0,0,0,11825,59,31,2,17
2,408906695083954177,1386325927,EvgeshaRe,"[идиотка, испугалась]",1,0,1,0,1273,26,27,0,15
3,408906695356973056,1386325927,ikonnikova_21,"[углу, сидит, погибает, голода, ещё, порции, в...",1,0,1,0,1549,19,17,0,35
4,408906761416867842,1386325943,JumpyAlex,"[значит, страшилка, блин, посмотрев, части, со...",1,0,0,0,597,16,23,1,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,409766329734795264,1386530880,Anastasievnaa,"[натааааш, классное, др, хрен, такое]",-1,0,0,0,2002,30,21,0,16
9996,409766375364644864,1386530891,EmoKingSize,"[ой, забыла, восстановить, старый, аккаунт, на...",-1,0,0,0,5154,41,49,0,23
9997,409766474010886144,1386530915,yegorushki,"[ночной, твиттер, спят, люблю, почитать, ночь,...",-1,0,0,0,1456,48,56,0,27
9998,409766520009793536,1386530926,blond_4,"[девочка, спасибо, тебе, вечер, очень, хватает...",-1,0,0,0,99,14,14,0,18


### 3.0 Токенизация